In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
#Kaggle Education Data import (Already Clean, Most columns have too much missing data) (2023)
educationData = pd.read_csv('Data/Raw/RawData.csv')
educationData = educationData[['Countries and areas',
                               'Birth_Rate', 
                               'Youth_15_24_Literacy_Rate_Male',
                               'Youth_15_24_Literacy_Rate_Female',
                               'Gross_Primary_Education_Enrollment', 
                               'Gross_Tertiary_Education_Enrollment',
                               'Unemployment_Rate']]
educationData = educationData.rename(columns={"Countries and areas": "Country"})

educationData['Average Youth Literacy Rate'] = (educationData['Youth_15_24_Literacy_Rate_Male'] + educationData['Youth_15_24_Literacy_Rate_Female'])/2

educationData = educationData[['Country',
                               'Birth_Rate', 
                               'Average Youth Literacy Rate',
                               'Gross_Primary_Education_Enrollment', 
                               'Gross_Tertiary_Education_Enrollment',
                               'Unemployment_Rate']]


In [3]:
# Wikipedia Table for Country Nominal GDP import and Clean up
def get_contents_from_web(url):
    tableData = []
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    tables = soup.find_all('table', class_='wikitable')
    
    for table in tables:
        rows = table.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            row_data = [ele.text.strip() for ele in cols]
            if len(row_data) > 6:
                tableData.append(row_data)
    
    return pd.DataFrame(tableData)

nominalGDP_data = get_contents_from_web('https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)')[1:]
dash = nominalGDP_data[4][192]

nominalGDP_data.drop(nominalGDP_data.loc[nominalGDP_data[2]== dash].index, inplace=True)
nominalGDP_data.drop(nominalGDP_data.loc[nominalGDP_data[4]== dash].index, inplace=True)
nominalGDP_data.drop(nominalGDP_data.loc[nominalGDP_data[6]== dash].index, inplace=True)
nominalGDP_data = nominalGDP_data.replace(',','', regex=True)

nominalGDP_data[2] = nominalGDP_data[2].astype(int)
nominalGDP_data[4] = nominalGDP_data[4].astype(int)
nominalGDP_data[6] = nominalGDP_data[6].astype(int)

nominalGDP_data.rename(columns={0: "Country"}, inplace=True)
nominalGDP_data.rename(columns={1: "Continent"}, inplace=True)
nominalGDP_data.rename(columns={2: "2023 Nominal GDP (USD Million)"}, inplace=True)
nominalGDP_data.rename(columns={4: "2022 Nominal GDP (USD Million)"}, inplace=True)
nominalGDP_data.rename(columns={6: "2021 Nominal GDP (USD Million)"}, inplace=True)

nominalGDP_data = nominalGDP_data[['Country', 'Continent',
                                   '2023 Nominal GDP (USD Million)', 
                                   '2022 Nominal GDP (USD Million)', 
                                   '2021 Nominal GDP (USD Million)']]
nominalGDP_data

,Country,Continent,2023 Nominal GDP (USD Million),2022 Nominal GDP (USD Million),2021 Nominal GDP (USD Million)
1,United States,Americas,26949643,25462700,23315081
2,China,Asia,17700899,17963171,17734131
3,Germany,Europe,4429838,4072192,4259935
4,Japan,Asia,4230862,4231141,4940878
5,India,Asia,3732224,3385090,3201471
...,...,...,...,...,...
190,Micronesia,Oceania,458,427,404
191,Marshall Islands,Oceania,277,280,257
193,Kiribati,Oceania,246,223,227
194,Nauru,Oceania,150,151,155


In [4]:
# Wikipedia Table for Nominal GDP per Capita import and Clean up
def get_contents_from_web2(url):
    tableData = []
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    tables = soup.find_all('table', class_='wikitable')
    
    for table in tables:
        rows = table.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            row_data = [ele.text.strip() for ele in cols]
            if len(row_data) > 7:
                tableData.append(row_data)
    
    return pd.DataFrame(tableData)

nominalGDP_PerCapita_data = get_contents_from_web2('https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)_per_capita')


nominalGDP_PerCapita_data.rename(columns={0: "Country"}, inplace=True)
nominalGDP_PerCapita_data.rename(columns={1: "Continent"}, inplace=True)
nominalGDP_PerCapita_data.rename(columns={2: "2023 GDP per Capita (USD)"}, inplace=True)
nominalGDP_PerCapita_data.rename(columns={4: "2022 GDP per Capita (USD)"}, inplace=True)
nominalGDP_PerCapita_data.rename(columns={6: "2021 GDP per Capita (USD)"}, inplace=True)

nominalGDP_PerCapita_data = nominalGDP_PerCapita_data[['Country', 'Continent',
                                                       "2023 GDP per Capita (USD)", 
                                                       "2022 GDP per Capita (USD)", 
                                                       "2021 GDP per Capita (USD)"]]

nominalGDP_PerCapita_data = nominalGDP_PerCapita_data.replace(',','', regex=True)
nominalGDP_PerCapita_data["2023 GDP per Capita (USD)"] = nominalGDP_PerCapita_data["2023 GDP per Capita (USD)"].astype(int)
nominalGDP_PerCapita_data["2022 GDP per Capita (USD)"] = nominalGDP_PerCapita_data["2022 GDP per Capita (USD)"].astype(int)
nominalGDP_PerCapita_data["2021 GDP per Capita (USD)"] = nominalGDP_PerCapita_data["2021 GDP per Capita (USD)"].astype(int)
nominalGDP_PerCapita_data


,Country,Continent,2023 GDP per Capita (USD),2022 GDP per Capita (USD),2021 GDP per Capita (USD)
0,Luxembourg,Europe,135605,126426,133745
1,Ireland,Europe,112248,104039,101109
2,Switzerland,Europe,102865,92102,93525
3,Norway,Europe,99266,106149,89242
4,Singapore,Asia,87884,82808,66822
...,...,...,...,...,...
189,Madagascar,Africa,530,505,500
190,South Sudan,Africa,417,1072,400
191,Sierra Leone,Africa,415,461,505
192,Afghanistan,Asia,443,364,373


In [5]:
#Combine All GDP data
totalGDP_data = pd.merge(nominalGDP_data, nominalGDP_PerCapita_data, on=['Country', 'Continent'])
totalGDP_data['Percentage growth from 2021 - 2023'] = ((totalGDP_data['2023 Nominal GDP (USD Million)'] - totalGDP_data['2021 Nominal GDP (USD Million)'])/totalGDP_data['2023 Nominal GDP (USD Million)'])*100
totalGDP_data['Per Capita Percentage growth from 2021 - 2023'] = ((totalGDP_data['2023 GDP per Capita (USD)'] - totalGDP_data['2021 GDP per Capita (USD)'])/totalGDP_data['2023 GDP per Capita (USD)'])*100
totalGDP_data

,Country,Continent,2023 Nominal GDP (USD Million),2022 Nominal GDP (USD Million),2021 Nominal GDP (USD Million),2023 GDP per Capita (USD),2022 GDP per Capita (USD),2021 GDP per Capita (USD),Percentage growth from 2021 - 2023,Per Capita Percentage growth from 2021 - 2023
0,United States,Americas,26949643,25462700,23315081,80412,76399,69185,13.486494,13.961846
1,China,Asia,17700899,17963171,17734131,12541,12720,12437,-0.187742,0.829280
2,Germany,Europe,4429838,4072192,4259935,52824,48433,51073,3.835422,3.314781
3,Japan,Asia,4230862,4231141,4940878,33950,33815,39650,-16.781828,-16.789396
4,India,Asia,3732224,3385090,3201471,2612,2389,2274,14.220824,12.940276
...,...,...,...,...,...,...,...,...,...,...
175,Micronesia,Oceania,458,427,404,4358,3741,3573,11.790393,18.012850
176,Marshall Islands,Oceania,277,280,257,6141,6728,6111,7.220217,0.488520
177,Kiribati,Oceania,246,223,227,1968,1702,1765,7.723577,10.315041
178,Nauru,Oceania,150,151,155,11757,10648,12390,-3.333333,-5.384027


In [6]:
#Combine into one dataset.
continentData1 = totalGDP_data.groupby('Continent').sum()[['2023 Nominal GDP (USD Million)',
                                                           '2022 Nominal GDP (USD Million)',
                                                           '2021 Nominal GDP (USD Million)']]

continentData2 = totalGDP_data.groupby('Continent').mean()[['2023 GDP per Capita (USD)',
                                                            '2022 GDP per Capita (USD)',
                                                            '2021 GDP per Capita (USD)',
                                                            'Percentage growth from 2021 - 2023', 
                                                            'Per Capita Percentage growth from 2021 - 2023']]

continentData = pd.merge(continentData1, continentData2, on=['Continent'])

combinedData = pd.merge(totalGDP_data[['Country', 
                        '2023 GDP per Capita (USD)',
                        '2022 Nominal GDP (USD Million)',
                        'Percentage growth from 2021 - 2023', 
                        'Per Capita Percentage growth from 2021 - 2023']], 
                        
                        educationData[['Country',
                                       'Birth_Rate',
                                       'Gross_Primary_Education_Enrollment',
                                       'Average Youth Literacy Rate',
                                       'Gross_Tertiary_Education_Enrollment',
                                       'Unemployment_Rate']],
                         on=['Country'])


In [7]:
#Saving all four datasets as CSV

from pathlib import Path  
filepath1 = Path('Data/Processed/GDP_Data.csv')  
filepath1.parent.mkdir(parents=True, exist_ok=True)  
totalGDP_data.to_csv(filepath1, index=False) 

filepath2 = Path('Data/Processed/Education_Data.csv')  
filepath2.parent.mkdir(parents=True, exist_ok=True)  
educationData.to_csv(filepath2, index=False) 

filepath3 = Path('Data/Processed/Combined_Data.csv')  
filepath3.parent.mkdir(parents=True, exist_ok=True)  
combinedData.to_csv(filepath3, index=False) 

filepath4 = Path('Data/Processed/Continent_Data.csv')  
filepath4.parent.mkdir(parents=True, exist_ok=True)  
continentData.to_csv(filepath4) 
